In [ ]:
import joblib

from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    roc_curve, auc, RocCurveDisplay
)

%matplotlib inline

In [ ]:
test_data = joblib.load('processed/test/data.joblib')
y_true = test_data[:, :6]

y_pred = joblib.load('processed/y_pred.joblib')
y_pred_class = y_pred.round()

y_true.shape, y_pred.shape

In [ ]:
# save plots
asset_dir = 'results/final/'
os.makedirs(asset_dir, exist_ok=True)

roc_auc_results = []
for i in range(6):
    filename = os.path.join(asset_dir, f'confusion_matrix_{i}.png')
    ConfusionMatrixDisplay(confusion_matrix(y_true[:, i], y_pred_class[:, i])).plot().figure_.savefig(filename)
    
    filename = os.path.join(asset_dir, f'roc_curve_{i}.png')
    fpr, tpr, thresholds = roc_curve(y_true[:, i], y_pred[:, i])
    roc_auc = auc(fpr, tpr)
    roc_auc_results.append(roc_auc)
    RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc).plot().figure_.savefig(filename)

In [ ]:
# print metrics
metrics = defaultdict(list)
for label in range(6):
    test_y_class_label = y_pred_class[:, label]
    predict_y_class = y_pred[:, label]
    predict_y_class_label = predict_y_class.round()

    metrics['accuracy'].append(accuracy_score(val_y_class, predict_y_class_label))
    metrics['precision'].append(precision_score(val_y_class, predict_y_class_label))
    metrics['recall'].append(recall_score(val_y_class, predict_y_class_label))
    metrics['roc_auc'].append(roc_auc_score(val_y_class, predict_y_class))

avg_metrics = {}
for name, results in metrics.items():
    print(f'{name} at class level: {results}')
    avg_metrics[name] = sum(results) / len(results)


results = [f'{name}: {result};' for name, result in avg_metrics.items()]
print(' '.join(results))